# --------------------------------------------------
# QUERY UNDERSTANDING
# --------------------------------------------------

In [79]:
# Prompt user to enter the query dynamically
# Ex: Generate a strategy intelligence report for the electric vehicle market and its key players
query = input("Please enter your query: ")

Please enter your query: Generate a strategy intelligence report for the electric vehicle market and its key players


In [80]:
# Define keyword-based industry mapping with sector-specific company lists
industry_keywords = {
    "electric vehicle": ["Tesla, Inc.", "Rivian Automotive, Inc.", "Lucid Group, Inc.", "NIO Inc."],
    "technology": ["Apple Inc.", "Microsoft Corporation", "Alphabet Inc.", "Amazon.com, Inc.", "Meta Platforms, Inc.", "NVIDIA Corporation", "Tesla, Inc."],
    "pharmaceutical": ["Pfizer Inc.", "Moderna, Inc.", "Johnson & Johnson", "AstraZeneca", "Bristol-Myers Squibb"],
    "cloud computing": ["Amazon Web Services", "Microsoft Azure", "Google Cloud", "IBM Cloud", "Salesforce"],
}

In [81]:
# Determine which sector to report on based on the query
selected_companies = []
for keyword, companies in industry_keywords.items():
    if keyword in query.lower():
        selected_companies = companies
        selected_sector = keyword.title()
        break

In [82]:
# Raise error if query doesn't match known sectors
if not selected_companies:
    raise ValueError("Error: No matching sector found for the query. Please refine your input.")

In [83]:
# Ticker symbol mapping for all supported companies
# This is used to fetch financial data via Yahoo Finance (yfinance)
ticker_map = {
    "Tesla, Inc.": "TSLA",
    "Rivian Automotive, Inc.": "RIVN",
    "Lucid Group, Inc.": "LCID",
    "NIO Inc.": "NIO",
    "Apple Inc.": "AAPL",
    "Microsoft Corporation": "MSFT",
    "Alphabet Inc.": "GOOGL",
    "Amazon.com, Inc.": "AMZN",
    "Meta Platforms, Inc.": "META",
    "NVIDIA Corporation": "NVDA",
    "Pfizer Inc.": "PFE",
    "Moderna, Inc.": "MRNA",
    "Johnson & Johnson": "JNJ",
    "AstraZeneca": "AZN",
    "Bristol-Myers Squibb": "BMY",
    "Amazon Web Services": "AMZN",
    "Microsoft Azure": "MSFT",
    "Google Cloud": "GOOGL",
    "IBM Cloud": "IBM",
    "Salesforce": "CRM"
}

# --------------------------------------------------
# DATA ACQUISITION
# --------------------------------------------------

In [84]:
# Import necessary libraries for scraping, finance data, and visualization
import requests
from bs4 import BeautifulSoup
import datetime
import matplotlib.pyplot as plt
import yfinance as yf

In [85]:
# Initialize storage for company data
all_data = []

In [86]:
# Fetch data for each selected company
for company in selected_companies:
    # Generate URL slug from company name to fetch data from companiesmarketcap.com
    search_name = company.replace(", Inc.", "").replace(".com", "").replace(" ", "-").replace(",", "").lower()
    url = f"https://companiesmarketcap.com/{search_name}/marketcap/"

    # Send request to market cap website
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')

    # Try scraping the company data from the marketcap page
    summary_box = soup.find("div", class_="company-summary-box")
    price_tag = summary_box.find(string=lambda t: "$" in t) if summary_box else None
    volume_tag = soup.find(string=lambda t: t and "Volume" in t)
    marketcap_tag = summary_box.find("div", class_="company-row").text if summary_box else "N/A"

    # Extract values or mark as N/A
    price = price_tag.strip() if price_tag else "N/A"
    volume = volume_tag.strip() if volume_tag else "N/A"
    marketcap = marketcap_tag.strip() if marketcap_tag else "N/A"

    # Fallback to Yahoo Finance if no data found or scraping fails
    if price == "N/A" or volume == "N/A" or marketcap == "N/A":
        ticker_symbol = ticker_map.get(company, None)
        if ticker_symbol:
            try:
                ticker = yf.Ticker(ticker_symbol)
                hist = ticker.history(period="1y")
                if not hist.empty:
                    pct_change = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
                    price = hist['Close'][-1]  # Last closing price
                    volume = hist['Volume'][-1]  # Last volume
                    marketcap = ticker.info.get("marketCap", "N/A")
                else:
                    pct_change = 0.0
                    price = "N/A"
                    volume = "N/A"
                    marketcap = "N/A"
            except Exception as e:
                pct_change = 0.0
                price = "N/A"
                volume = "N/A"
                marketcap = "N/A"
        else:
            pct_change = 0.0

    # Append structured data for reporting
    all_data.append({
        "name": company,
        "price": price,
        "volume": volume,
        "marketcap": marketcap,
        "1yr_change": round(pct_change, 2)
    })

<ipython-input-86-271e40d64e66>:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  pct_change = ((hist['Close'][-1] - hist['Close'][0]) / hist['Close'][0]) * 100
<ipython-input-86-271e40d64e66>:31: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  price = hist['Close'][-1]  # Last closing price
<ipython-input-86-271e40d64e66>:32: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  volume = hist['Volume'][-1]  # Last volume
<ipython-input-86-27

# --------------------------------------------------
# DATA VISUALIZATION
# --------------------------------------------------

In [87]:
# Prepare data for plotting
names = [d['name'] for d in all_data]
prices = [float(d['price']) if "$" in str(d['price']) else 0 for d in all_data]
changes = [d['1yr_change'] for d in all_data]

In [88]:
# Generate and save a bar chart of stock prices
plt.figure(figsize=(10, 5))
plt.bar(names, prices, color='skyblue')
plt.title(f'Stock Prices of Key Players in {selected_sector} Sector')
plt.xticks(rotation=45)
plt.ylabel("Price (USD)")
plt.tight_layout()
plt.close()

# --------------------------------------------------
# REPORT WRITING
# --------------------------------------------------

In [89]:
# Capture current date for the report
today = datetime.date.today().strftime("%B %d, %Y")

In [90]:
# Print report structure
print("=" * 50)
print(f"INDUSTRY INTELLIGENCE REPORT: {selected_sector} Sector")
print(f"Generated on: {today}")
print("=" * 50)
print("1. MARKET OVERVIEW")
print("-" * 30)
print("- Companies analyzed: " + ", ".join(names))
print()
print("2. COMPETITOR ANALYSIS")
print("-" * 30)
for d in all_data:
    print(f"{d['name']}:")
    print(f"  - Stock Price: {d['price']}")
    print(f"  - Trading Volume: {d['volume']}")
    print(f"  - Market Cap: {d['marketcap']}")
    print(f"  - 1-Year Change: {d['1yr_change']}%\n")

print("3. STRATEGIC RECOMMENDATIONS")
print("-" * 30)
print("1. Invest in companies with consistent 1-year growth and strong market capitalization.")
print("2. Diversify into newer entrants with high potential upside.")
print("3. Closely watch high-volume stocks for short-term opportunities.")
print("-" * 30)
print("END OF REPORT")
print("=" * 50)

INDUSTRY INTELLIGENCE REPORT: Electric Vehicle Sector
Generated on: April 19, 2025
1. MARKET OVERVIEW
------------------------------
- Companies analyzed: Tesla, Inc., Rivian Automotive, Inc., Lucid Group, Inc., NIO Inc.

2. COMPETITOR ANALYSIS
------------------------------
Tesla, Inc.:
  - Stock Price: 241.3699951171875
  - Trading Volume: 83404800
  - Market Cap: 776371372032
  - 1-Year Change: 60.99%

Rivian Automotive, Inc.:
  - Stock Price: 11.600000381469727
  - Trading Volume: 16103300
  - Market Cap: 13116236800
  - 1-Year Change: 31.37%

Lucid Group, Inc.:
  - Stock Price: 2.380000114440918
  - Trading Volume: 68394700
  - Market Cap: 7214970368
  - 1-Year Change: -2.46%

NIO Inc.:
  - Stock Price: 3.5199999809265137
  - Trading Volume: 23168700
  - Market Cap: 7913453056
  - 1-Year Change: -12.0%

3. STRATEGIC RECOMMENDATIONS
------------------------------
1. Invest in companies with consistent 1-year growth and strong market capitalization.
2. Diversify into newer entrants 